In [1]:
import os
import pandas as pd
import spacy
from nltk.corpus import stopwords
import string
from gensim import corpora
import gensim
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

/home/ubuntu/PyTorch/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-09-09 11:36:47.737684: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-09 11:36:47.763666: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-09 11:36:47.763820: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
file = os.getcwd()+'/Data/papers.csv'
data = pd.read_csv(file, nrows=10)


In [3]:
data.head(2)

,id,year,title,event_type,pdf_name,abstract,paper_text
0,1,1987,Self-Organization of Associative Database and ...,NaN,1-self-organization-of-associative-database-an...,Abstract Missing,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...
1,10,1987,A Mean Field Theory of Layer IV of Visual Cort...,NaN,10-a-mean-field-theory-of-layer-iv-of-visual-c...,Abstract Missing,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...


In [4]:
documents = data['paper_text']
documents

def clean_documents(documents):
    nlp = spacy.load('en_core_web_sm')
    stops = stopwords.words('english')
    alphabet = list(string.ascii_lowercase)
    stops = stops + alphabet
    
    
    cleaned_documents = []
    for doc in documents:
        doc = nlp(doc)
        tokens = []
        for token in doc:
            if token.tag_ in ['NNP', 'VBZ','VBG', 'IN'] and str(token.text).lower() not in stops and len(token.text)>2:
                tokens.append(str(token.lemma_).lower())
        cleaned_documents.append(tokens)
    return clean_documents

In [5]:
documents = clean_documents(documents)

In [6]:
id2word = corpora.Dictionary(documents)

TypeError: 'function' object is not iterable

In [ ]:
id2word

In [ ]:
print(id2word)

In [ ]:
tokens = id2word.token2id
tokens[:10]

In [ ]:
def idtoword(cleaned_documents):
    corpus = []
    
    for text in cleaned_documents:
        new = id2word.doc2bow(text)
        corpus.append(new)
    return corpus

In [ ]:
corpus = idtoword(cleaned_documents)

In [ ]:
model = gensim.models.ldamodel.LdaModel(
        corpus = corpus,
        id2word=id2word,
        num_topics=5,
        random_state=100,
        update_every =1,
        chunksize=100,
        passes=10,
        alpha='auto')
    
vis = gensimvis.prepare(
        model, 
        corpus, 
        id2word, 
        mds="mds",
        R=30)

In [ ]:
pyLDAvis.save_html(vis, os.getcwd()+'/Results/ldavisbigramstrigrams.html')

In [ ]:
def cluster_test(corpus, model):
    docs_with_1_topic = 0
    docs_with_multiple_topics = 0
    docs_with_no_topics = 0
    total_docs = 0
    for doc in corpus:
        topics = model.get_document_topics(doc, minimum_probability=0.20)
        total_docs += 1
        if len(topics) == 1:
            docs_with_1_topic += 1
        elif len(topics) > 1:
            docs_with_multiple_topics += 1
        else:
            docs_with_no_topics += 1
    print('Corpus assigned to a single topic:', (docs_with_1_topic / total_docs) * 100, '%')
    print('Corpus assigned to multiple topics:', (docs_with_multiple_topics / total_docs) * 100, '%')
    print('corpus assigned to no topics:', (docs_with_no_topics / total_docs) * 100, '%')

In [ ]:
cluster_test(corpus, model)

In [ ]:
def retrieval_test(new_doc, lda, index):
    new_bow = dictionary.doc2bow(new_doc)  # change new document to bag of words representation
    new_vec = lda[new_bow]  # change new bag of words to a vector
    index.num_best = 10  # set index to generate 10 best results
    matches = (index[new_vec])
    scores = []
    for match in matches:
        score = (match[1])
        scores.append(score)
        score = str(score)
        key = 'doc_' + str(match[0])
        article_dict = doc2metadata[key]
        author = article_dict['author']
        title = article_dict['title']
        year = article_dict['pub_year']
        print(key + ': ' + author.title() + ' (' + year + '). ' + title.title() + '\n\tsimilarity score -> ' + score + '\n')

In [7]:
file = os.getcwd()+'/Data/papers.csv'
data = pd.read_csv(file, nrows=20)

In [8]:
data = data['paper_text']

In [9]:
data.head(2)

0    767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...
1    683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...
Name: paper_text, dtype: object

In [10]:
from final import pipeline_LDA

pipeline_LDA(data, 4)

Documents Cleaned
Fitting the LDA Model


/home/ubuntu/PyTorch/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


Results saved in Results Folder


/home/ubuntu/PyTorch/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/ubuntu/PyTorch/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/ubuntu/PyTorch/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/ubuntu/PyTorch/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/ubuntu/PyTorch/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the i